In [0]:
!pip install konlpy # konlpy 패키지 설치

In [0]:
room = ["바닥은 삐걱거리는 나무판자로 되어있다.",\
        "바닥은 차갑다",\
        "방 가운데에는 돌로 된 탁자가 하나 놓여있다.",\
        "방에 문이 하나 있다.",\
        "문은 나무로 되어있다.",\
        "문은 열리지 않는다.",\
        "문에는 잠금장치가 걸려있다.",\
        "방에 침대가 하나 있다.",\
        "탁자 옆에는 의자가 두 개 놓여있다.",\
        "침대 밑에는 열쇠가 있다.",\
        "방에 창문이 없다."]

# 형태소 단위 토큰화
---


In [24]:
from konlpy.tag import Okt # 속도가 빠르지만 정확하게 분류해내지 못함.
o = Okt()
room_pos = o.pos(room[0])
room_nouns = o.nouns(room[0])
print(room_pos)
print(room_nouns)

[('바닥', 'Noun'), ('은', 'Josa'), ('삐걱', 'Noun'), ('거리', 'Noun'), ('는', 'Josa'), ('나무', 'Noun'), ('판자', 'Noun'), ('로', 'Josa'), ('되어있다', 'Verb'), ('.', 'Punctuation')]
['바닥', '삐걱', '거리', '나무', '판자']


In [26]:
from konlpy.tag import Kkma # 속도가 느리지만 정확하게 분류해낸다.
k = Kkma()
room_pos = k.pos(room[0])
room_nouns = k.nouns(room[0])
print(room_pos)
print(room_nouns)

[('바닥', 'NNG'), ('은', 'JX'), ('삐걱거리', 'VV'), ('는', 'ETD'), ('나무', 'NNG'), ('판자', 'NNG'), ('로', 'JKM'), ('되', 'VV'), ('어', 'ECD'), ('있', 'VXV'), ('다', 'EFN'), ('.', 'SF')]
['바닥', '나무', '나무판자', '판자']


# 어간 추출(Stemming) 및 표제어 추출(Lemmatization)
---



 - 뿌리 단어들을 찾아가서 단어의 개수를 줄일 수 있는지 확인
 - 예) am, are, is -> be
 - 형태학적 파싱이 필요.
 - 형태소의 두 가지 종류
    1. 어간(stem) : 단어의 의미를 담고 있는 핵심 부분.
    2. 접사(affix) : 단어에 추가적인 의미를 주는 부분.

# 불용어(stopwords) 제거
---

- 조사, 접속사 제거
- 직접 정의 후 제거

In [30]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [0]:
# 직접 정의 후 제거
stop_words = "된 있다 되어있다 놓여있다 ."
stop_words = stop_words.split(' ')
tk_list = []
for i in range(len(room)):
  tk_list.append(word_tokenize(room[i]))

result = []
for tk in tk_list:
  for w in tk:
    if w not in stop_words:
      result.append(w)

In [55]:
print(result)

['바닥은', '삐걱거리는', '나무판자로', '바닥은', '차갑다', '방', '가운데에는', '돌로', '탁자가', '하나', '방에', '문이', '하나', '문은', '나무로', '문은', '열리지', '않는다', '문에는', '잠금장치가', '걸려있다', '방에', '침대가', '하나', '탁자', '옆에는', '의자가', '두', '개', '침대', '밑에는', '열쇠가', '방에', '창문이', '없다']


# 정수 인코딩
---

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(result)

In [61]:
print(tokenizer.word_index) # 빈도 수가 높은 순서대로 index를 부여

{'하나': 1, '방에': 2, '바닥은': 3, '문은': 4, '삐걱거리는': 5, '나무판자로': 6, '차갑다': 7, '방': 8, '가운데에는': 9, '돌로': 10, '탁자가': 11, '문이': 12, '나무로': 13, '열리지': 14, '않는다': 15, '문에는': 16, '잠금장치가': 17, '걸려있다': 18, '침대가': 19, '탁자': 20, '옆에는': 21, '의자가': 22, '두': 23, '개': 24, '침대': 25, '밑에는': 26, '열쇠가': 27, '창문이': 28, '없다': 29}


In [62]:
print(tokenizer.texts_to_sequences(tk_list))

[[3, 5, 6], [3, 7], [8, 9, 10, 11, 1], [2, 12, 1], [4, 13], [4, 14, 15], [16, 17, 18], [2, 19, 1], [20, 21, 22, 23, 24], [25, 26, 27], [2, 28, 29]]


In [0]:
# 빈도수 상위 5개 단어만 인코딩
vocab_size = 5
tokenizer = Tokenizer(num_words=vocab_size+1)
tokenizer.fit_on_texts(tk_list)

In [66]:
tokenizer.texts_to_sequences(tk_list)

[[5, 1],
 [5],
 [2, 1],
 [3, 2, 4, 1],
 [1],
 [1],
 [1],
 [3, 2, 4, 1],
 [1],
 [4, 1],
 [3, 1]]

# 원-핫 인코딩
---

- 한계
  - 벡터를 저장하기 위한 공간이 계속해서 늘어남
  - 단어의 유사도 표현이 불가

In [0]:
from konlpy.tag import Okt
okt = Okt()
token = okt.morphs(' '.join(room))

In [71]:
word2index = {} # 토큰마다 고유 인덱스 부여
for voca in token:
  if voca not in word2index.keys():
    word2index[voca] = len(word2index)
print(word2index)

{'바닥': 0, '은': 1, '삐걱': 2, '거리': 3, '는': 4, '나무': 5, '판자': 6, '로': 7, '되어있다': 8, '.': 9, '차갑다': 10, '방': 11, '가운데': 12, '에는': 13, '돌': 14, '된': 15, '탁자': 16, '가': 17, '하나': 18, '놓여있다': 19, '에': 20, '문': 21, '이': 22, '있다': 23, '열': 24, '리지': 25, '않는다': 26, '잠금장치': 27, '걸려있다': 28, '침대': 29, '옆': 30, '의자': 31, '두': 32, '개': 33, '밑': 34, '열쇠': 35, '창문': 36, '없다': 37}


In [0]:
def one_hot_encoding(word, word2index):
  one_hot_vector = [0]*(len(word2index))
  index = word2index[word]
  one_hot_vector[index] = 1
  return one_hot_vector

In [75]:
print(one_hot_encoding("침대",word2index))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [81]:
# 케라스를 이용한 one hot encoding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
tokenizer = Tokenizer()
tokenizer.fit_on_texts(room)
print(tokenizer.word_index)

{'하나': 1, '방에': 2, '있다': 3, '바닥은': 4, '되어있다': 5, '놓여있다': 6, '문은': 7, '삐걱거리는': 8, '나무판자로': 9, '차갑다': 10, '방': 11, '가운데에는': 12, '돌로': 13, '된': 14, '탁자가': 15, '문이': 16, '나무로': 17, '열리지': 18, '않는다': 19, '문에는': 20, '잠금장치가': 21, '걸려있다': 22, '침대가': 23, '탁자': 24, '옆에는': 25, '의자가': 26, '두': 27, '개': 28, '침대': 29, '밑에는': 30, '열쇠가': 31, '창문이': 32, '없다': 33}


In [83]:
sub_text = "침대가 나무로 되어있다."
encoded = tokenizer.texts_to_sequences([sub_text])[0]
print(encoded)

[23, 17, 5]


In [84]:
one_hot = to_categorical(encoded)
one_hot

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

# 단어 분리(Subword Segmentation)
---

- BPE(Byte pair Encoding) 알고리즘, WPM(Word Piece Model) 등이 있다.
- 하나의 단어가 의미있는 여러 단어의 조합인 경우가 많기 때문에 효과적